# Лабораторная работа 5. Временные ряды. 


Результат лабораторной работы &mdash; отчет. Мы предпочитаем принимать отчеты в формате ноутбуков IPython (ipynb-файл). Постарайтесь сделать ваш отчет интересным рассказом, последовательно отвечающим на вопросы из заданий. Помимо ответов на вопросы, в отчете так же должен быть код, однако чем меньше кода, тем лучше всем: нам &mdash; меньше проверять, вам &mdash; проще найти ошибку или дополнить эксперимент. При проверке оценивается четкость ответов на вопросы, аккуратность отчета и кода.


### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу &mdash; 10 баллов. Сдавать задание после указанного в lk срока сдачи нельзя. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов и понижают карму (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий в открытом источнике, необходимо прислать ссылку на этот источник (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, нам необходима ссылка на источник).



In [ ]:
import os
import numpy as np
import pandas as pd
from zipfile import ZipFile

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(palette='Set2')

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import statsmodels.api as sm
import pmdarima as pm
from tbats import TBATS

## 0. Разминка

#### Задача 1. [2 балла]

Даны временные ряды
* $y_t = 2 + y_{t-1} - 0.5 y_{t-2} + 0.5 y_{t-3} + \varepsilon_t - 2\varepsilon_{t-1} + \varepsilon_{t-2}$,
* $y_t = -1 - y_{t-2} - 0.25 y_{t-4} + \varepsilon_t - \varepsilon_{t-3}$,

где $\varepsilon_t$ &mdash; гауссовский белый шум со средним 0 и дисперсией $\sigma^2$, который так же не зависит от $y_{t-i}, i \geqslant 1$. 

**1.** Являются ли ряды стационарными? 

**2.** Определите тип процесса в терминах ARIMA($p, d, q$).

**3.** Если временной ряд стационарен, вычислите математическое ожидание и дисперсию $y_t$.

## 1. Данные

В качестве данных рассмотрим известный набор данных с параметрами погоды в одной точке земной поверхности, доступный нам с частотой дискретизации в 10 минут. Описание данных доступно по <a href="https://keras.io/examples/timeseries/timeseries_weather_forecasting/">ссылке</a>.

Скачаем данные

In [ ]:
uri = "https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip"
!wget {uri}

In [ ]:
zip_path = "jena_climate_2009_2016.csv.zip"
assert os.path.exists(zip_path), f"Download {uri}"

zip_file = ZipFile(zip_path)
zip_file.extractall()
csv_path = "jena_climate_2009_2016.csv"

Немного преобразуем данные, рассмотрев максимальное значение каждой величины за сутки.

In [ ]:
weather_df = pd.read_csv(csv_path)
weather_df["Date"] = pd.to_datetime(weather_df["Date Time"], format="%d.%m.%Y %H:%M:%S")

weather_df_daily = weather_df.set_index("Date").resample('D').max().reset_index()
del weather_df_daily["Date Time"]
weather_df_daily = weather_df_daily.set_index('Date')
weather_df_daily.head()

Теперь в каждой строчке датасета мы имеем максимальные значения за сутки. Например, в строчке `2012-12-22` в колонке `T (degC)` записана максимальная температура по Цельсию за данные сутки.

**Глобальная цель** в данном домашнем задании &mdash; прогнозирование максимальной температуры на день вперед. Например, 22 декабря нужно построить прогноз значения `T (degC)` за 23 декабря, используя при этом любые данные до 22 декабря включительно, но не используя никакие данные начиная с 23 декабря.

*Замечание*. Обратите внимание, что значения от 22 декабря доступны только по окончанию этого дня, плюс накладные расходы на поставку данных. Но для упрощения ситуации, допустим, что данные приходят мгновенно, а прогноз от нас ждут в начале суток 23 декабря, так что мы успеем подучить данные от 22 декабря, преобразовать их в признаки и применить нашу модель.

#### Задача 2. [0.25 балла]

Выделите таргет в отдельную переменную `pd.Series`. Значением таргета от 22 декабря должна быть максимальная температура за сутки 23 декабря, то есть то, что мы хотим прогнозировать.

Разделите данные на обучающие и тестовые. В качестве тестового отрезка времени возьмите данные за последний год. Обратите внимание, что за 2016-10-26 и 2016-10-27 данных нет.

## 2. Простая аналитика

#### Задача 3. [1 балл]

Произведите визуализацию всех временных рядов.

Имеются ли в данных выбросы? Если да, отфильтруйте данные. Не перестарайтесь, фильтруйте только более менее очевидные выбросы.

Подумайте над тем, какие значения ставить вместо выбросов. Учтите, что не все модели могут обрабатывать пропуски.

#### Задача 4. [1 балл]

Посчитайте матрицу корреляций по всем временным рядам без сдвигов (т.е. $corr(x_t, y_t)$) и визуализируйте ее

Постройте коррелограмму временного ряда температуры

По временному ряду температуры выполните STL-разложение и визуализируйте полученные компоненты

**Выводы:**

## 3. Качество

#### Задача 5. [0.5 балла]

Напишите функции вычисления качества решения в задачи прогнозирования.
Качество решения будем описывать при помощи нескольких метрик качства:
  * mean_squared_error
  * mean_absolute_error
  * mean_absolute_percentage_error
  * median_absolute_error

По всем данным посчитайте метрики качества baseline-решения &mdash; прогноз константой (т.е. прогноз последним доступным значением самой прогнозируемой величины), для горизонтов прогнозирования 1, 2, 3.

Посчитайте эти же метрики по скользящим окнам из 4 и 8 недель. Иначе говоря, 
* сначала считаете качество по окну $[t+1, t+\Delta t]$, используя в качестве прогноза $y_t$,
* затем по окну $[t+2, t+\Delta t+1]$, используя в качестве прогноза $y_{t+1}$,
* и так далее.

Постройте графики изменения качества прогноза со временем.

**Выводы:**

#### Задача 6. [0.5 балла]

Скользящая валидация с фиксированным размером обучающего периода (*см. в лекциях вариант 1*) реализована в <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html">sklearn.model_selection.TimeSeriesSplit</a>:

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
ts_cv_1 = TimeSeriesSplit(n_splits=5, max_train_size=1000, test_size=8*7, gap=70)

Если вы будете валидировать свои модели по данной схеме, то вы сможете получить наилучшую модель, но не сможете получить несмещённую оценку её качества.
Это важно когда заказчик хочет не только самую лучшую модель по качеству, но и оценку качества этой модели на исторических данных.

Для этого случая имеется альтернативный nested-способ валидации, реализацию которого можно найти по <a href="https://github.com/roelbertens/time-series-nested-cv">ссылке</a>.

Реализуйте nested кросс-валидацию.

## 4. Сведение к задаче регрессии

Для получения признакового описания для конкретного момента времени обычно применяют различные оконные функции к таблице данных, что позволяет сэкономить память и вычисления.
Но нужно помнить, что нельзя допустить просачивание информации из будущего, т.к. такие признаки вы не сможете вычислить в продакшне.




#### Задача 7. [0.5 балла]

Вычислите простые признаки вида описательных статистик (сумма, максимум, минимум, стандартное отклонение) по нескольким размерам окон. Признаки можно вычислять не только по температуре, но и по сопутствующим временным рядам.

#### Задача 8. [0.5 балла]

Обучите градиентый бустинг и обычный многослойный перцептрон с использованием написанного скользящего контроля. Посчитайте качество.

Постройте визуализируйте полученные прогнозы на графике, сравнив их с истинными значениями ряда. Отдельно постройте также график тестового интервала с целью увеличения масштаба графика.

#### Задача 9. [1 балл]

Генерировать признаки на этапе знакомства с данными &mdash; не очень интересное занятие.
В отрасли предпринято несколько попыток автоматизировать этот этап.

С помощью библиотеки <a href="https://tsfresh.com/">tsfresh</a> можно довольно быстро сгенерировать большое количество признаков на основе сырых данных временных рядов. Например, по скользящему окну с помощью можно вычислить статистики вида минимум, максимум, среднее, количество пиков и т.д.

Ознакомьтесь с данной библиотекой и сгенерируйте признаковое описание. Обратите внимание на функции
* `roll_time_series`
* `extract_features`

Сколько признаков вы получили? Как побороться с большим числом признаков?

Обучите модель, посчитайте ее качество и постройте график прогноза.

**Выводы:**

## 5. ARIMA-подобные модели

#### Задача 10. [1.5 балла]

Обучите модель ARIMA, 
* используя сопутствующие временные ряды в качестве экзогенных факторов,
* используя знания о сезонности,
* подобрав оптимальные гиперпараметры.

*Замечания.*
* Используйте `pmdarima.auto_arima`, которая автоматически подберет гиперпараметры, *см. семинар*.
* Обучение может быть *слишком* вычислительно затратным. Начинайте тестировать код на очень маленьких данных, затем постепенно увеличивайте размер данных.
* Обучение встроенной сезонности может не получиться. В таком случае можно попробовать передать значения за предыдущие сезоны как экзогенные факторы, или же моделировать сезонность рядами Фурье. 

Обучите модель, посчитайте ее качество и постройте график прогноза *вместе с предсказательным интервалом*.

Напечатайте саммари модели (`summary()`). Что и как вы можете проинтерпретировать?

#### Задача 11. [1 балл]

Обучите модель TBATS, используя знания о сезонности. Выполните те же операции, что и для ARIMA.

**Выводы:**

## 6. Итоговое сравнение

#### Задача 12. [0.25 балла]

Составьте итоговую таблицу сравнения качества прогнозов всех рассмотренных моделей по всем метрикам. Постройте график прогнозов для лучших моделей.

**Выводы:**